In [1]:
import math
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import lxml
import re
from random import seed
from collections import Counter
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import array
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

C:\Users\MoJo\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LinkIndoor800m = "http://www.ustfccca.org/records-lists/outdoor-collegiate-all-time-bests?gender=1&sport=1&div=20527&top=25&event=6"

In [3]:
page = requests.get(LinkIndoor800m)
soup = BeautifulSoup(page.text,'html')
soup

table = soup.find("table")

In [4]:
Header = soup.findAll('th')
Headers = []
for h in Header:
    Headers.append(h.text)

In [5]:
Headers

['Athlete', 'Institution', 'Mark', 'Date', '']

In [6]:
row_soup = soup.findAll('tr')

In [7]:
row_soup

[<tr><th style="width: 35%;">Athlete</th><th colspan="2" style="width: 30%;">Institution</th><th style="width: 25%;">Mark</th><th style="width: 10%;">Date</th><th></th></tr>,
 <tr><td colspan="100" style="font-weight: bolder;">UNOFFICIAL Top 25 Performers and Performances</td></tr>,
 <tr><td colspan="5"><div class="container-table" style="width: 100%;"><table class="embed" width="100%"><thead><tr></tr></thead><tr><td style="width: 35%;"><span>(1) Paul ERENG</span><span style="float:right;font-size:0.9em;">SO</span></td><td style="width: 5%;"><span class="group-logo-inline"><img alt="Virginia" class="group-logo-inline" src="/images/logos/rankings-article/virginia.png" title="Virginia"/></span></td><td style="width: 25%;"><a href="/records-lists/outdoor-collegiate-all-time-bests?gender=1&amp;sport=1&amp;div=20527&amp;top=25&amp;event=6&amp;team=1699">Virginia</a></td><td style="width: 25%;">1:44.84<a href="https://www.youtube.com/watch?v=u-724HNIYJI" target="_blank">📺</a> </td><td style=

In [8]:
rows = []
for row in row_soup:
    row_data = []
    for cell in row.findAll('td'):
        row_data.append(cell.text)
    rows.append(row_data)

In [9]:
rows

[[],
 ['UNOFFICIAL Top 25 Performers and Performances'],
 ['(1) Paul ERENGSOVirginia1:44.84📺 3/4/1989',
  '(1) Paul ERENGSO',
  '',
  'Virginia',
  '1:44.84📺 ',
  '3/4/1989'],
 [],
 ['(1) Paul ERENGSO', '', 'Virginia', '1:44.84📺 ', '3/4/1989'],
 ['(2) Michael SARUNISOUTEP1:45.15📺 3/10/2018\xa0\xa0\xa0\xa0\xa053.11/62.04, last 200: 26.17',
  '(2) Michael SARUNISO',
  '',
  'UTEP',
  '1:45.15📺 ',
  '3/10/2018',
  '\xa0\xa0\xa0\xa0\xa053.11/62.04, last 200: 26.17'],
 [],
 ['(2) Michael SARUNISO', '', 'UTEP', '1:45.15📺 ', '3/10/2018'],
 ['\xa0\xa0\xa0\xa0\xa053.11/62.04, last 200: 26.17'],
 ['\xa0\xa0\xa0\xa0— Saruni {2}SOUTEP1:45.19📺 bb = Performance occurred on a 200m/220yd banked track 1/27/2018',
  '\xa0\xa0\xa0\xa0— Saruni {2}SO',
  '',
  'UTEP',
  '1:45.19📺 bb = Performance occurred on a 200m/220yd banked track ',
  '1/27/2018'],
 [],
 ['\xa0\xa0\xa0\xa0— Saruni {2}SO',
  '',
  'UTEP',
  '1:45.19📺 bb = Performance occurred on a 200m/220yd banked track ',
  '1/27/2018'],
 ['(3) Brando

In [10]:
len(rows)

125

In [11]:
rows[123]

['Thomas STAINESJR',
 '',
 'CSU Pueblo',
 '1:46.27 OTOT = Performance occurred on an oversized-track (>200 meters/220 yards) ',
 '2/8/2019']

In [12]:
len(rows)

125

In [13]:
RealRows = []

In [14]:
for i in range(0,124):
    if len(rows[i]) > 1:
        RealRows.append(rows[i])
        

In [15]:
RealRows

[['(1) Paul ERENGSOVirginia1:44.84📺 3/4/1989',
  '(1) Paul ERENGSO',
  '',
  'Virginia',
  '1:44.84📺 ',
  '3/4/1989'],
 ['(1) Paul ERENGSO', '', 'Virginia', '1:44.84📺 ', '3/4/1989'],
 ['(2) Michael SARUNISOUTEP1:45.15📺 3/10/2018\xa0\xa0\xa0\xa0\xa053.11/62.04, last 200: 26.17',
  '(2) Michael SARUNISO',
  '',
  'UTEP',
  '1:45.15📺 ',
  '3/10/2018',
  '\xa0\xa0\xa0\xa0\xa053.11/62.04, last 200: 26.17'],
 ['(2) Michael SARUNISO', '', 'UTEP', '1:45.15📺 ', '3/10/2018'],
 ['\xa0\xa0\xa0\xa0— Saruni {2}SOUTEP1:45.19📺 bb = Performance occurred on a 200m/220yd banked track 1/27/2018',
  '\xa0\xa0\xa0\xa0— Saruni {2}SO',
  '',
  'UTEP',
  '1:45.19📺 bb = Performance occurred on a 200m/220yd banked track ',
  '1/27/2018'],
 ['\xa0\xa0\xa0\xa0— Saruni {2}SO',
  '',
  'UTEP',
  '1:45.19📺 bb = Performance occurred on a 200m/220yd banked track ',
  '1/27/2018'],
 ['(3) Brandon MILLERSOTexas A&M1:45.24 2/26/2022',
  '(3) Brandon MILLERSO',
  '',
  'Texas A&M',
  '1:45.24 ',
  '2/26/2022'],
 ['(3) Bran

In [16]:
RealRealRows = RealRows[::2]
RealRealRows

[['(1) Paul ERENGSOVirginia1:44.84📺 3/4/1989',
  '(1) Paul ERENGSO',
  '',
  'Virginia',
  '1:44.84📺 ',
  '3/4/1989'],
 ['(2) Michael SARUNISOUTEP1:45.15📺 3/10/2018\xa0\xa0\xa0\xa0\xa053.11/62.04, last 200: 26.17',
  '(2) Michael SARUNISO',
  '',
  'UTEP',
  '1:45.15📺 ',
  '3/10/2018',
  '\xa0\xa0\xa0\xa0\xa053.11/62.04, last 200: 26.17'],
 ['\xa0\xa0\xa0\xa0— Saruni {2}SOUTEP1:45.19📺 bb = Performance occurred on a 200m/220yd banked track 1/27/2018',
  '\xa0\xa0\xa0\xa0— Saruni {2}SO',
  '',
  'UTEP',
  '1:45.19📺 bb = Performance occurred on a 200m/220yd banked track ',
  '1/27/2018'],
 ['(3) Brandon MILLERSOTexas A&M1:45.24 2/26/2022',
  '(3) Brandon MILLERSO',
  '',
  'Texas A&M',
  '1:45.24 ',
  '2/26/2022'],
 ['(4) Devin DIXONJRTexas A&M1:45.27📺 bb = Performance occurred on a 200m/220yd banked track 1/26/2019\xa0\xa0\xa0\xa0\xa024.32, 26.21 [50.53], 27.18, 27.57 [54.75]',
  '(4) Devin DIXONJR',
  '',
  'Texas A&M',
  '1:45.27📺 bb = Performance occurred on a 200m/220yd banked track 

In [17]:
RealRealRows[1][4]

'1:45.15📺 '

In [18]:
len(RealRealRows)

37

In [19]:
OfficialTimes = []

In [20]:
for i in range(0,29):
    OfficialTimes.append(RealRealRows[i][4])
OfficialTimes

['1:44.84📺 ',
 '1:45.15📺 ',
 '1:45.19📺 bb = Performance occurred on a 200m/220yd banked track ',
 '1:45.24 ',
 '1:45.27📺 bb = Performance occurred on a 200m/220yd banked track ',
 '1:45.33  ',
 '1:45.59 ',
 '1:45.80  ',
 '1:45.88  ',
 '1:45.90 ',
 '1:45.90 bb = Performance occurred on a 200m/220yd banked track ',
 '1:45.91 ',
 '1:45.93 bb = Performance occurred on a 200m/220yd banked track ',
 '1:45.95  ',
 '1:45.98 ',
 '1:45.98  ',
 '1:46.02📺 ',
 '1:46.05 ',
 '1:46.05  ',
 '1:46.06 ',
 '1:46.07 ',
 '1:46.08 ',
 '1:46.08 bb = Performance occurred on a 200m/220yd banked track ',
 '1:46.09  ',
 '1:46.13  ',
 '1:46.19  ',
 '1:46.20 bb = Performance occurred on a 200m/220yd banked track ',
 '1:46.28 ',
 '1:46.28  ']

In [21]:
RealRealRows[0][1]

'(1) Paul ERENGSO'

In [22]:
Names = []

for i in range(0,29):
    Names.append(RealRealRows[i][1])
Names

['(1) Paul ERENGSO',
 '(2) Michael SARUNISO',
 '\xa0\xa0\xa0\xa0— Saruni {2}SO',
 '(3) Brandon MILLERSO',
 '(4) Devin DIXONJR',
 '(5) Patrick NDUWIMANASR',
 '(6) Charlie HUNTERJR',
 '(7) Einars TUPURITIS',
 '(8) Derrick PETERSONJR',
 '\xa0\xa0\xa0\xa0— Hunter {2}JR',
 '(9) Marco AROPSO',
 '(10) Finley MCLEARSO',
 '(11) Donavan BRAZIERFR',
 '(12) Dirk HEINZE',
 '(13) Takieddine HEDEILLISR',
 '\xa0\xa0\xa0\xa0— Tupuritis {2}',
 '(14) Yusuf BIZIMANASO',
 '(15) Edward KEMBOI',
 '(15) Trinity GRAY',
 '(17) Sam AUSTINSO',
 '(18) Leoman MOMOHSR',
 '(19) Isaiah HARRISJR',
 '\xa0\xa0\xa0\xa0— Brazier {2}FR',
 '(20) Jackson LANGAT',
 '(21) Otukile LEKOTEFR',
 '(22) George KERSH',
 '(23) Andres ARROYOJR',
 '(24) Crayton CARROZZAJR',
 '(24) Mark EVERETT']

In [23]:
OfficialReGexNames = []

In [24]:
for i in range(0,29):
    if Names[i].startswith("\xa0\xa0"):
        OfficialReGexNames.append(Names[i][4:])
    else:
        OfficialReGexNames.append(Names[i])
        
OfficialReGexNames

['(1) Paul ERENGSO',
 '(2) Michael SARUNISO',
 '— Saruni {2}SO',
 '(3) Brandon MILLERSO',
 '(4) Devin DIXONJR',
 '(5) Patrick NDUWIMANASR',
 '(6) Charlie HUNTERJR',
 '(7) Einars TUPURITIS',
 '(8) Derrick PETERSONJR',
 '— Hunter {2}JR',
 '(9) Marco AROPSO',
 '(10) Finley MCLEARSO',
 '(11) Donavan BRAZIERFR',
 '(12) Dirk HEINZE',
 '(13) Takieddine HEDEILLISR',
 '— Tupuritis {2}',
 '(14) Yusuf BIZIMANASO',
 '(15) Edward KEMBOI',
 '(15) Trinity GRAY',
 '(17) Sam AUSTINSO',
 '(18) Leoman MOMOHSR',
 '(19) Isaiah HARRISJR',
 '— Brazier {2}FR',
 '(20) Jackson LANGAT',
 '(21) Otukile LEKOTEFR',
 '(22) George KERSH',
 '(23) Andres ARROYOJR',
 '(24) Crayton CARROZZAJR',
 '(24) Mark EVERETT']

In [25]:
CleanedNames = []

for i in range(0,len(OfficialReGexNames)):
    if OfficialReGexNames[i].startswith("—"):
        CleanedNames.append(OfficialReGexNames[i][2:])
    if OfficialReGexNames[i].startswith("("):
        CleanedNames.append(OfficialReGexNames[i][4:])        
CleanedNames

['Paul ERENGSO',
 'Michael SARUNISO',
 'Saruni {2}SO',
 'Brandon MILLERSO',
 'Devin DIXONJR',
 'Patrick NDUWIMANASR',
 'Charlie HUNTERJR',
 'Einars TUPURITIS',
 'Derrick PETERSONJR',
 'Hunter {2}JR',
 'Marco AROPSO',
 ' Finley MCLEARSO',
 ' Donavan BRAZIERFR',
 ' Dirk HEINZE',
 ' Takieddine HEDEILLISR',
 'Tupuritis {2}',
 ' Yusuf BIZIMANASO',
 ' Edward KEMBOI',
 ' Trinity GRAY',
 ' Sam AUSTINSO',
 ' Leoman MOMOHSR',
 ' Isaiah HARRISJR',
 'Brazier {2}FR',
 ' Jackson LANGAT',
 ' Otukile LEKOTEFR',
 ' George KERSH',
 ' Andres ARROYOJR',
 ' Crayton CARROZZAJR',
 ' Mark EVERETT']

In [26]:
CleanedNames2 = []

for i in range(0,len(OfficialReGexNames)):
    if CleanedNames[i].startswith(" "):
        CleanedNames2.append(CleanedNames[i][1:])
    else:
        CleanedNames2.append(CleanedNames[i])        
CleanedNames2

['Paul ERENGSO',
 'Michael SARUNISO',
 'Saruni {2}SO',
 'Brandon MILLERSO',
 'Devin DIXONJR',
 'Patrick NDUWIMANASR',
 'Charlie HUNTERJR',
 'Einars TUPURITIS',
 'Derrick PETERSONJR',
 'Hunter {2}JR',
 'Marco AROPSO',
 'Finley MCLEARSO',
 'Donavan BRAZIERFR',
 'Dirk HEINZE',
 'Takieddine HEDEILLISR',
 'Tupuritis {2}',
 'Yusuf BIZIMANASO',
 'Edward KEMBOI',
 'Trinity GRAY',
 'Sam AUSTINSO',
 'Leoman MOMOHSR',
 'Isaiah HARRISJR',
 'Brazier {2}FR',
 'Jackson LANGAT',
 'Otukile LEKOTEFR',
 'George KERSH',
 'Andres ARROYOJR',
 'Crayton CARROZZAJR',
 'Mark EVERETT']

In [27]:
len(OfficialTimes)

29

In [28]:
len(OfficialReGexNames)

29

In [29]:
Schools = []

for i in range(0,len(OfficialTimes)):
    Schools.append(RealRealRows[i][3])
Schools

['Virginia',
 'UTEP',
 'UTEP',
 'Texas A&M',
 'Texas A&M',
 'Arizona',
 'Oregon',
 'Wichita State',
 'Missouri',
 'Oregon',
 'Mississippi State',
 'Miami (Ohio)',
 'Texas A&M',
 'Arkansas',
 'Texas Tech',
 'Wichita State',
 'Texas',
 'Iowa State',
 'Brown',
 'Florida',
 'Arkansas',
 'Penn State',
 'Texas A&M',
 'TCU',
 'South Carolina',
 'Ole Miss',
 'Florida',
 'Texas',
 'Florida']

In [30]:
Times = []
for i in range(0,len(OfficialTimes)):
    Times.append(OfficialTimes[i][:7])
    
Times

['1:44.84',
 '1:45.15',
 '1:45.19',
 '1:45.24',
 '1:45.27',
 '1:45.33',
 '1:45.59',
 '1:45.80',
 '1:45.88',
 '1:45.90',
 '1:45.90',
 '1:45.91',
 '1:45.93',
 '1:45.95',
 '1:45.98',
 '1:45.98',
 '1:46.02',
 '1:46.05',
 '1:46.05',
 '1:46.06',
 '1:46.07',
 '1:46.08',
 '1:46.08',
 '1:46.09',
 '1:46.13',
 '1:46.19',
 '1:46.20',
 '1:46.28',
 '1:46.28']

In [31]:
MensIndoor800nDF = pd.DataFrame({'Names': CleanedNames2, 'Times': Times, 'Schools': Schools})
MensIndoor800nDF.to_csv('MensIndoor800m.csv', index=False)